In [12]:
%reset -f
### Varian Multileaf Collimator (MLC) treatment plan data editor ###
# author: Petr Bruza, Dartmouth College, 12/2015
# reference: MLC File Format Description Reference Guide, P/N 1106064-05, July 2013
# notes: optimized for Varian Millenium 120-leaf MLC

# ----- input headers & scan parameters -----
# -----           EDIT HERE             -----

Filename = 'DoseOptics-Dynamic_revH.mlc'

Header = {
    'File Rev': 'H',
    'Treatment': 'Dynamic Dose', #Static / Dynamic Dose
    'Last Name': 'Optics',
    'First Name': 'Dose',
    'Patient ID': 'vms_DoseOptics',#vms_ heading defines a "dummy" patient in DHMC's DICOM 
    'Number of Fields': 0, #>2 for dose dynamic
    'Model': 'Varian 120M',
    'Tolerance': 0.05, #[cm] position tolerance of the leaves during dynamic treatment mode. If leaf position outside tolerance, beam is halted. 0.05 - 0.50 for dose dynamic, 0.1 - 1 for arc dynamic 
    'Field': 0,
    'Index': 0, #for dose dynamic - dose fraction for the control point, from 0.0000 to 1.0000
    'Carriage Group': 1,
    'Operator': 'Petr Bruza',
    'Collimator Orientation': [0], # must be a list: [], collimator orientation: source-to-isocenter view - 0° towards gantry bearing, clockwise
    'Collimator': 0,
    'Note': '0',
    'Shape': '0',
    'Magnification': '1.00',
    'CRC': 0
}
Field_Params = { # scan parameters
    'Sheet Width': 5, # [integer] defines width of the scanned area by number of scanning leaves. other leaves are closed
    'Leaf Gap': 0.5, # [mm] defines width of the sheet beam
    'Scan Distance': 50, # [mm] total distance travelled by leaves
    'Scan Step': 0.5, # [mm] distance travelled by leaves between two consecutive steps, also step distance
    'Longitudinal Offset': 0, # [mm] offset of scanned area along leaf motion direction
    'Lateral Offset': 0 # [integer] offset of scanned area perpendicular to leaf motion, defined by no. of leavess
}
# -------------------------------------------


import numpy as np
import codecs
from operator import itemgetter

def collect(l, index):
    return map(itemgetter(index), l)

# CRC calculation functions (_initial, _update_crc, crc) by Serge Ballesta 2014
# http://www.lammertbies.nl/comm/info/crc-calculation.html, 

CRC_Polynomial = 0x1021
CRC_Preset = 0xFFFF

def _initial(c):
    crc = 0
    c = c << 8
    for j in range(8):
        if (crc ^ c) & 0x8000:
            crc = (crc << 1) ^ CRC_Polynomial
        else:
            crc = crc << 1
        c = c << 1
    return crc

_tab = [ _initial(i) for i in range(256) ]

def _update_crc(crc, c):
    cc = 0xff & c
    tmp = (crc >> 8) ^ cc
    crc = (crc << 8) ^ _tab[tmp & 0xff]
    crc = crc & 0xffff
    return crc

def crc(str):
    crc = CRC_Preset
    for c in str:
        crc = _update_crc(crc, ord(c))
    return crc

# sets proper coding (UTF-8) and heading (no BOM for rev. A-H, BOM-8 for rev. J) of different revisions of the output file
if Header['File Rev'] == 'J':
    MLC_File = codecs.open(Filename, 'w','utf-8') # wipes the plan of same filename without asking!! to be removed
    MLC_File.write(u'\ufeff')# UTF-8 BOM required header according to MLC File Rev J
else:
    MLC_File = codecs.open(Filename, 'w','utf-8')
    MLC_File.close()

# Static plan switch - for static, only one field is generated - it is neccesary to override the number of scans
if Header['Treatment'] == 'Static':
    Field_Params['Scan Step'] = Field_Params['Scan Distance']

# returns A or B leaf string by calling index 0 or 1 (later used as LUT from Leaf_Array index X)
Leaf_AB = ['A', 'B'] 

# required header keyword order (Table 2, MLC manual)
Header_Keys = ['File Rev', 'Treatment', 'Last Name', 'First Name', 'Patient ID', 'Number of Fields', 'Model', 'Tolerance']
Leading_Field_Keys = ['Field', 'Index', 'Carriage Group', 'Operator', 'Collimator']
Trailing_Field_Keys = ['Note', 'Shape', 'Magnification']

# calculates total number of "fields" and leaf indices from given scanning parameters
Header['Number of Fields'] = (int(Field_Params['Scan Distance']/Field_Params['Scan Step'])) * len(Header['Collimator Orientation'])
First_Scan_Leaf = int(30 -Field_Params['Sheet Width']/2 +Field_Params['Lateral Offset'])# index of first leaf / defines the beginning of sheet beam
Scanning_Leaves = np.arange(First_Scan_Leaf,First_Scan_Leaf + Field_Params['Sheet Width'],1)# indices of leaves that are moving

#treatment plan/scan matrix pre-allocation, uniform dose distribution along all scans
Leaf_Array = np.zeros((2, 60, Header['Number of Fields']/len(Header['Collimator Orientation'])))
Scan_Array = np.zeros((1, 60, Header['Number of Fields']/len(Header['Collimator Orientation'])))
Dose_Value_List = np.arange(0.0000,1.0000,1./(Header['Number of Fields']))# +1 for zero dose in first step
Dose_Value_List = np.append(Dose_Value_List,1)# adds the final fractional dose =1 (leading 0 iz for MLC homing only)

#list of individual leaf gap offsets* from MLC center axis* (* .. better explanation in labbook)
Scan_Plane_Positions = np.arange(-Field_Params['Scan Distance']/2. +Field_Params['Longitudinal Offset'] +Field_Params['Scan Step']/2.,Field_Params['Scan Distance']/2. +Field_Params['Longitudinal Offset'] +Field_Params['Scan Step']/2.,Field_Params['Scan Step'])

#projection of leaf gap offsets* over all scanning leafs
for Leaf_No in Scanning_Leaves:
    Scan_Array[0, Leaf_No, :] =  Scan_Plane_Positions
    Leaf_Array[0, Leaf_No, :] =  Scan_Plane_Positions + Field_Params['Leaf Gap']/2.
    Leaf_Array[1, Leaf_No, :] = -Scan_Plane_Positions + Field_Params['Leaf Gap']/2.

#writes file header
for key in Header_Keys:
    with codecs.open(Filename, 'a', 'utf-8') as MLC_File:
        MLC_File.write('%s = %s\r\n' %(key, str(Header[key])))
        print('%s = %s' %(key, str(Header[key])))

with codecs.open(Filename, 'a', 'utf-8') as MLC_File:
    MLC_File.write('\r\n') #blank line

#writes individual treatment plan fields: first 
Field_Index = 0
with codecs.open(Filename, 'a', 'utf-8') as MLC_File:
    
    for angle in Header['Collimator Orientation']: # for-loop scans through all collimator orientations
 
        
        if Header['Treatment'] == 'Dynamic Dose':
            Header['Index'] = '%.4f' %0 
            for key in Leading_Field_Keys:
                MLC_File.write('%s = %s\r\n' %(key, str(Header[key])))
            
            #writes first field with zero index and zero dose
            for index, value in np.ndenumerate(Leaf_Array[:,:,0]):
                MLC_File.write('Leaf %2s%s =%8.3f\r\n' %(index[1]+1, Leaf_AB[index[0]], value))#writes Leaf<number><bank>=<value>, number + 1 because inices in python begins with 0
            MLC_File.write('%s = %s\r\n' %(key, str(Header[key])))
            MLC_File.write('\r\n')
            Field_Index = Field_Index +1
      
        for field, position in np.ndenumerate(Leaf_Array[1, 1, :]): # for-loop runs through all scanning fields (Z-axis in Leaf_Array). field and position extracted from enumerated list of MLC leaves
            Header['Index'] = '%.4f' % Dose_Value_List[Field_Index] # defines Index header as a fractional dose with given precision 
            Header['Field'] = Field_Index # defines Field header
            Field_Index += 1
            
            # writes leading header for each field
            for key in Leading_Field_Keys:
                MLC_File.write('%s = %s\r\n' %(key, str(Header[key])))

            # scans through enumerated Leaf_Array list, returns indices and corresp. values of enumerated entries in Leaf_Array list 
            # (original X,Y positions, e.g. A/B leaf (X), Leaf index (Y, 0-59))
            for index, value in np.ndenumerate(Leaf_Array[:,:,field[0]]):
                MLC_File.write('Leaf %2s%s =%8.3f\r\n' %(index[1]+1, Leaf_AB[index[0]], value))##writes Leaf<number><bank>=<value>, number + 1 because inices in python begins with 0

            # writes trailing header for each field
            for key in Trailing_Field_Keys:
                MLC_File.write('%s = %s\r\n' %(key, str(Header[key])))
            MLC_File.write('\r\n') # writes a blank line after each field

# reads created output file and calculates CRC
with codecs.open(Filename, 'rb', 'utf-8') as MLC_File:
    data = MLC_File.read().replace('\r\n', '') #neccesary to EOL with \r\n (Windows EOL)
    Header['CRC'] = crc(data)
    #print(format(Header['CRC'], '04X'))

# appends CRC to the output file
with codecs.open(Filename, 'a', 'utf-8') as MLC_File:
    MLC_File.write('%s = %s\r\n' %('CRC', format(Header['CRC'], '04X')))# str(Header['CRC']))) 


File Rev = H
Treatment = Dynamic Dose
Last Name = Optics
First Name = Dose
Patient ID = vms_DoseOptics
Number of Fields = 100
Model = Varian 120M
Tolerance = 0.05
